In [1]:
%cd /home/nahuel/ecg/generalization/models
%load_ext autoreload
%autoreload 2

/home/nahuel/ecg/generalization/models


/home/nahuel/ecg/generalization/generalization/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import sys
import os

# Agregar la carpeta que contiene 'data.py' al sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../data')))

In [3]:
from model import select_model, ECGModel, evaluate, fine_tune
from data import load_data, get_shot

Data

In [5]:
dataset_name = "MIT-toy"

train_dataloader, val_dataloader, test_dataloader, _ = load_data(dataset_name)

Model

In [7]:
model_name = "CNN"
model_module = select_model(model_name)

path = "/home/nahuel/ecg/generalization/models/saved/lightning_logs/version_1/checkpoints/best_model.ckpt"
model = ECGModel.load_from_checkpoint(path, model=model_module)

Evaluation

In [17]:
device = "cpu"

print("Evaluation on MIT-BIH")
evaluate(model, test_dataloader, device)

Evaluation on MIT-BIH
Accuracy: 0.9794
F1 Score: 0.8948


Fine-tuning

Primero escoger shot=5, 10, 50. Es decir, 5 samples por clase, 10 samples por clase...

In [12]:
import numpy as np

dataset = train_dataloader.dataset
class_counts = np.unique([label for _, label in dataset], return_counts=True)[1]
# Número de clases en tu dataset
num_classes = len(class_counts)

# Crear subconjuntos para 50, 100, 150 muestras
subset_5 = get_shot(dataset, shot=5, num_classes=num_classes)
subset_10 = get_shot(dataset, shot=10, num_classes=num_classes)
subset_50 = get_shot(dataset, shot=50, num_classes=num_classes)

In [32]:
fine_tune(model_name, model, subset_5)

model.conv1.weight: False
model.conv1.bias: False
model.conv2.weight: False
model.conv2.bias: False
model.conv3.weight: False
model.conv3.bias: False
model.conv4.weight: False
model.conv4.bias: False
model.conv5.weight: False
model.conv5.bias: False
model.conv6.weight: False
model.conv6.bias: False
model.conv7.weight: False
model.conv7.bias: False
model.conv8.weight: False
model.conv8.bias: False
model.fc1.weight: True
model.fc1.bias: True
model.fc2.weight: True
model.fc2.bias: True
model.fc3.weight: True
model.fc3.bias: True
Epoch 1/10, Loss: 0.6024
Epoch 2/10, Loss: 0.3030
Epoch 3/10, Loss: 0.5283
Epoch 4/10, Loss: 0.5303
Epoch 5/10, Loss: 0.2561
Epoch 6/10, Loss: 0.6047
Epoch 7/10, Loss: 0.5209
Epoch 8/10, Loss: 0.4964
Epoch 9/10, Loss: 0.4889
Epoch 10/10, Loss: 0.4234


In [35]:
# Evidentemente esto no tiene sentido, porque estoy re-entrenando con
# los mismos datos que ya usé para entrenar el modelo. Pero es un ejemplo.
evaluate(model, test_dataloader, device)

Accuracy: 0.9819
F1 Score: 0.9065
